# Planar simple shear

Relevant resources:
- Figure 5 in [Kaminski & Ribe, 2001](https://doi.org/10.1016%2Fs0012-821x%2801%2900356-9)
- Figure 3 in [Lev & Hager, 2008](https://doi.org/10.1029%2F2008gc002032)

## Pure olivine

Shear defined by $\frac{\partial v_{y}}{\partial x}$ so that the active slip system is $(010)[100]$ as in [Zhang & Karato, 1995](https://doi.org/10.1038%2F375774a0).

In [1]:
import numpy as np
from scipy import linalg as la
from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation

from pydrex import minerals as _minerals
from pydrex import deformation_mechanism as _defmech
from pydrex import diagnostics as _diagnostics
from pydrex import stats as _stats
from pydrex import logger as _log

In [2]:
# Set up parameters
n_grains = 3000
gbm_mobilities = [0, 50, 200]
params = {
    "olivine_fraction": 1,
    "stress_exponent": 3.5,
    "deformation_exponent": 1.5,
    "gbs_threshold": 0,
    "nucleation_efficiency": 5,
    "number_of_grains": n_grains,
}

In [3]:
# Set up initial conditions
rng = np.random.default_rng(seed=8816)

velocity_gradient = np.array([[0, 0, 0], [2, 0, 0], [0, 0, 0]])
deformation_gradient_init = np.eye(3)
orientations_init = Rotation.from_rotvec(
    [[0, 0, -θ * np.pi / 2] for θ in rng.random(n_grains)]
).as_matrix()


def create_mineral():
    return _minerals.Mineral(
        _minerals.MineralPhase.olivine,
        _minerals.OlivineFabric.A,
        _defmech.Regime.dislocation,
        fractions_init=np.full(n_grains, 1 / n_grains),
        orientations_init=orientations_init,
    )

In [4]:
%%capture
# Set up output figure.

fig = plt.figure(constrained_layout=True)
ax0 = fig.add_subplot(3, 1, 1)
ax50 = fig.add_subplot(3, 1, 2, sharex=ax0)
ax200 = fig.add_subplot(3, 1, 3, sharex=ax0)

ax0.tick_params(labelbottom=False)
ax50.tick_params(labelbottom=False)
ax50.set_xlabel("strain ($D_{0}t$)")
ax200.set_ylabel("mean angle of [100] axes to shear plane")

In [8]:
# Compute CPO and FSE angles for M* = {0, 50, 200}.
for ax, gbm_mobility in zip((ax0,ax50,ax200), gbm_mobilities):
    params["gbm_mobility"] = gbm_mobility

    timestamps = np.linspace(0, 200, 500)
    θ_fse = np.empty_like(timestamps)
    θ_cpo = np.empty_like(timestamps)
    mineral = create_mineral()
    deformation_gradient = deformation_gradient_init.copy()

    for t, time in enumerate(timestamps[:-1]):
        deformation_gradient = mineral.update_orientations(
            params,
            deformation_gradient,
            velocity_gradient,
            pathline=(time, timestamps[t + 1], np.array([0, 0, 0])),
        )

        # Compute FSE angle using principal strains.
        fse_λ, fse_v = la.eigh(
            deformation_gradient @ deformation_gradient.transpose()
        )
        λ3, λ2, λ1 = np.sqrt(fse_λ)
        fse_orientation = Rotation.from_matrix(fse_v[:, ::-1])
        θ_fse[t] = _diagnostics.smallest_angle(
            fse_orientation.apply([λ1, 0, 0]), [0, 1, 0]
        )

        # Compute CPO angle using Bingham average.
        orientations_resampled, _ = _stats.resample_orientations(
            mineral.orientations[-1], mineral.fractions[-1],
        )
        direction_mean = _diagnostics.bingham_average(
            orientations_resampled,
            axis=_minerals.OLIVINE_PRIMARY_AXIS[mineral.fabric],
        )
        θ_cpo[t] = _diagnostics.smallest_angle(
            direction_mean, [0, 1, 0]
        )

    ax.plot(timestamps, θ_fse, "--", label="FSE")
    ax.plot(timestamps, θ_cpo, label="CPO")
    ax.legend(title=f"$M^{{\ast}}={gbm_mobility}$")

fig.show()

INFO [07:30] pydrex: created Mineral(phase=0, fabric=0, regime=1, n_grains=1000, fractions=<list of ndarray (3000,)>, orientations=<list of ndarray (3000, 3, 3)>)
INFO [07:30] pydrex: calculating CPO at [0, 0, 0] (t=0.000000e+00) with velocity gradient [0, 0, 0, 2, 0, 0, 0, 0, 0]
rv_cb_arr is NULL
Call-back cb_f_in_lsoda__user__routines failed.


ValueError: 0-th dimension must be fixed to 30009 but got 10009
